<a href="https://colab.research.google.com/github/Ahm1027/Payloads-Model/blob/master/Payload_Model_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from google.colab import drive
import binascii

In [22]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [225]:
globalFeatures = np.array([])

In [226]:
def resolveMalwareDirs(FOLDER_PATHS, num_of_features=2000):
  dict = {}
  for files in os.listdir(FOLDER_PATHS):
    filepath = r"/content/drive/MyDrive/Payloads/"+files
    data = get_hexdump(filepath)
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:num_of_features]
    return mergeWithGlobalVocab(data_sorted, globalFeatures)

globalFeatures = resolveMalwareDirs(r"/content/drive/MyDrive/Payloads")
globalFeatures


2000
2000


array([['20202020', '6c617373', '636c6173', ..., '31253735', '3127292c',
        '31272c20'],
       ['937', '578', '577', ..., '12', '12', '12']], dtype='<U32')

In [174]:
globalFeatures

In [15]:
def get_hexdump(executable):
  executable = open(executable, 'rb')
  data = executable.read()
  executable.close()
  data2hex = bin2hex(data)
  output = generate_ngrams(text=data2hex, WordsToCombine=4)
  return output

def bin2hex(data):
  data2hex = ""
  for byte in data:
    hex = str(binascii.hexlify(bytes([byte]))).split("'")
    if (len(hex) >= 2):
      data2hex += hex[1]+" "
    else:
      data2hex += hex[0]+" "
  return data2hex

def generate_ngrams(text, WordsToCombine):
    words = text.split()
    output = []  
    for i in range(len(words)- WordsToCombine+1):
      word = ''
      for i in range (i, i+WordsToCombine):
        word = word + words[i]
      output.append(word)
    return output

# def calculateTotalCounts(uniqueOutput, output):
#   Uniquecounts = []
#   for i in range(len(uniqueOutput)):
#     count = 0
#     for j in range(len(output)):
#       if output[j] == uniqueOutput[i]:
#         count = count+1
#     Uniquecounts.append(count)
#   return Uniquecounts

In [42]:
def vectorize(data):
  vectorizer = CountVectorizer()
  vectorizer.fit(data)
  vectorized = vectorizer.transform(data)
  return vectorizer, vectorized

In [136]:
def arrangeAndSort(data_vectorizer, data_vectorized):
  data_arranged = list(zip(data_vectorizer.get_feature_names(), data_vectorized.sum(0).getA1()))
  data_sorted = sorted(temp, key=lambda item: item[1], reverse=True)
  return data_sorted

In [128]:
def normalizeFeatures(data):
  data_scaled = (data - data.min()) / (data.max() - data.min())
  return data_scaled

In [227]:
def find_first_no_jit(item, vec):
  for i in range(len(vec)):
    if item == vec[i]:
      return i
  return -1

def mergeWithGlobalVocab(data, globalFeatures):
  if (len(globalFeatures) == 0):
    globalFeatures = np.concatenate([globalFeatures, np.array(data)[:, 0].astype(str).tolist()])
  else:
    globalFeatures[0] = np.concatenate([globalFeatures, np.array(data)[:, 0].astype(str).tolist()])
    globalFeatures[0], indexes = np.unique(globalFeatures, return_index=True)
    globalFeatures[0] = globalFeatures[np.sort(indexes)]
  features = [0]*len(globalFeatures)
  for x in data:
    id = find_first_no_jit(x[0], globalFeatures)
    if (id != -1):
      features[id] = x[1]
  globalFeatures = np.vstack((globalFeatures, features))
  return globalFeatures


In [ ]:
compressed_SVD = TruncatedSVD(30)
compressed_data = compressed_SVD.fit_transform(features_frequency)
pd.DataFrame(compressed_SVD.components_)